<a href="https://colab.research.google.com/github/Cerion73/Coursera/blob/main/RNN_Tutorial_(Text_Processing).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
import tensorflow as tf
from keras.api.utils import image_dataset_from_directory
import cv2
import numpy as np
import matplotlib.pyplot as plt

img_dir = 'images'
img_data = tf.data.Dataset.from_tensor_slices()

# for image_path in os.listdir(img_dir):
#   img = cv2.imread(os.path.join(img_dir, image_path))
#   img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img_data

AttributeError: type object 'DatasetV2' has no attribute 'data_from_directory'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [7]:
tf.expand_dims([[2, 4],[2, 7]], 0)

<tf.Tensor: shape=(1, 2, 2), dtype=int32, numpy=
array([[[2, 4],
        [2, 7]]], dtype=int32)>

In [8]:
url = "https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"

dataset = tf.keras.utils.get_file("aclImdb_v1", url,
                                    untar=True, cache_dir='.',
                                    cache_subdir='')

dataset_dir = os.path.join(os.path.dirname(dataset), 'aclImdb')

84125825/84125825 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step


In [9]:
dataset_dir

'./aclImdb'

In [11]:
os.listdir(os.path.join('aclImdb_v1', dataset_dir))

['test', 'train', 'imdbEr.txt', 'README', 'imdb.vocab']

In [12]:
train_dir = 'train'
test_dir = 'test'

os.listdir(os.path.join('aclImdb_v1', dataset_dir, train_dir))

['unsupBow.feat',
 'urls_unsup.txt',
 'neg',
 'labeledBow.feat',
 'unsup',
 'pos',
 'urls_pos.txt',
 'urls_neg.txt']

In [16]:
import shutil
train_path = os.path.join('aclImdb_v1', dataset_dir, train_dir)
test_path = os.path.join('aclImdb_v1', dataset_dir, test_dir)

remover_dir = os.path.join(train_path, 'unsup')
shutil.rmtree(remover_dir)
os.listdir(train_path)

['unsupBow.feat',
 'urls_unsup.txt',
 'neg',
 'labeledBow.feat',
 'pos',
 'urls_pos.txt',
 'urls_neg.txt']

In [17]:
max_features = 100
batch_size = 32

train_ds_raw = tf.keras.utils.text_dataset_from_directory(
    train_path,
    batch_size=batch_size,
    validation_split=0.2,
    subset='training',
    seed=42
)

val_ds_raw = tf.keras.utils.text_dataset_from_directory(
    train_path,
    batch_size=batch_size,
    validation_split=0.2,
    subset='validation',
    seed=42
)

test_ds_raw = tf.keras.utils.text_dataset_from_directory(
    train_path,
    batch_size=batch_size
)

Found 25000 files belonging to 2 classes.
Using 20000 files for training.
Found 25000 files belonging to 2 classes.
Using 5000 files for validation.
Found 25000 files belonging to 2 classes.


In [32]:
import re
import string


def custom_standardizer(input_data):
  lower_case = tf.strings.lower(input_data)
  stripped_html = tf.strings.regex_replace(lower_case, '<br />', ' ')
  return tf.strings.regex_replace(stripped_html,
                                  '[%s]' % re.escape(string.punctuation), '')


vectorizing_layer = tf.keras.layers.TextVectorization(
    standardize=custom_standardizer,
    max_tokens=max_features,
    output_mode='int',
    output_sequence_length=max_features
)
train_text = train_ds_raw.map(lambda text, labels: text)
vectorizing_layer.adapt(train_text)

def vectorize_data(data, labels):
  text_data = tf.expand_dims(data, -1)
  return vectorizing_layer(data), labels



In [33]:
train_b= next(iter(train_ds_raw))
train_batch, label_batch = train_b
first_review, first_label = train_batch[0], label_batch[0]
print("Review", first_review)
print("Label", first_label)
print("Vectorized review: ----", vectorize_data(first_review, first_label))

Review tf.Tensor(b"My children watch the show everyday that its on. Its a great program for younger children. However they need to stop showing re-runs and do some more actual shows and get rid of Rooney's and Deedee's YELLOW TEETH. Moe is the only Doodle bop with clean white pearlie teeth and the children notice these things and ask if the 2 don't ever brush their teeth? Does the show ever make its way to the United States and if so where can we find its schedule at. And one other thing if we might be able to add. Moe you need to stop hiding so much. Sometimes when you pop up out of no where you scare the younger children and whats with the pulling of the rope? What does that signify? other then getting wet all the time. They need to add newer things to their show instead of the same ole same ole. Kids loose interest that way.", shape=(), dtype=string)
Label tf.Tensor(0, shape=(), dtype=int32)
Vectorized review: ---- (<tf.Tensor: shape=(100,), dtype=int64, numpy=
array([54,  1,  1,  2

In [34]:
train_ds = train_ds_raw.map(vectorize_data)
val_ds = val_ds_raw.map(vectorize_data)
test_ds = test_ds_raw.map(vectorize_data)



In [38]:
from keras.api.layers import Embedding, GlobalAveragePooling1D, Dense, Dropout
from keras.api. models import Sequential

embedding_dim = 16

model = Sequential([
    Embedding(max_features + 1, embedding_dim),
    Dropout(0.2),
    GlobalAveragePooling1D(),
    Dropout(0.2),
    Dense(1, activation='sigmoid')
])

model.compile(loss=tf.losses.BinaryCrossentropy, optimizer='adam', metrics=[tf.metrics.BinaryAccuracy(threshold=0.5)])



In [40]:
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10
)

Epoch 1/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 8s 13ms/step - binary_accuracy: 0.6783 - loss: 0.5948 - val_binary_accuracy: 0.6708 - val_loss: 0.5974
Epoch 2/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 11s 15ms/step - binary_accuracy: 0.6769 - loss: 0.5948 - val_binary_accuracy: 0.6702 - val_loss: 0.6020
Epoch 3/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 14s 22ms/step - binary_accuracy: 0.6763 - loss: 0.5968 - val_binary_accuracy: 0.6716 - val_loss: 0.5975
Epoch 4/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 13s 11ms/step - binary_accuracy: 0.6752 - loss: 0.5939 - val_binary_accuracy: 0.6712 - val_loss: 0.5979
Epoch 5/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 12s 14ms/step - binary_accuracy: 0.6753 - loss: 0.5954 - val_binary_accuracy: 0.6712 - val_loss: 0.5997
Epoch 6/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 6s 8ms/step - binary_accuracy: 0.6766 - loss: 0.5949 - val_binary_accuracy: 0.6736 - val_loss: 0.5977
Epoch 7/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 9s 7ms/step - binary_accuracy: 0.6746 - loss: 0.5942 - val_binary_accuracy: 0.6722 - val_loss: 0.59

In [41]:
connected_model = tf.keras.Sequential([
    vectorizing_layer,
    model,
    tf.keras.layers.Activation('sigmoid')
])

